In [1]:
from ultralytics import YOLO
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os

## LOAD MODEL

In [2]:
model = YOLO('./model/client_model/best.pt')
model2 = YOLO('./model/client_model/best.pt')

## READ FROM VIDEO

In [3]:
video_path = './dataset/Video/sample.mp4'
cap = cv2.VideoCapture(video_path)

## READ FROM WEBCAM

In [4]:
# cap = cv2.VideoCapture(0) 

## LICENSE DETECTION

In [5]:
captured = []
len_file = os.listdir('./license_predict/')
plate = len(len_file)

In [6]:
def create_detection_frame(frame):
    new_height = int(frame.shape[0] * 0.4)  # 40% dari tinggi frame asli
    new_width = frame.shape[1]  # Lebar sesuai dengan frame asli

    # Hitung koordinat untuk rectangle
    top_left_x = 0
    top_left_y = frame.shape[0] - new_height - int(frame.shape[0] * 0.1)  # Angkat rectangle ke tengah 10%
    bottom_right_x = frame.shape[1]
    bottom_right_y = top_left_y + new_height
    
    cv2.rectangle(frame, (top_left_x, top_left_y), (bottom_right_x, bottom_right_y), (0, 255, 255), thickness=2)
    
    return frame[top_left_y:bottom_right_y, top_left_x:bottom_right_x], top_left_x, top_left_y, bottom_right_x, bottom_right_y

In [7]:
def detect_objects(frame, model):
    # Lakukan deteksi objek menggunakan model
    result = model.track(frame, persist=True)
    
    if result is not None and result[0].boxes.id is not None:
        track_ids = result[0].boxes.id.int().cpu().tolist()
        boxes = result[0].boxes.xywh.cpu()
        frame_ = result[0].plot()   
        
        return frame_, track_ids, result, boxes
    else:
        return None, None, None, None

In [8]:
def detect_license_plate(detected_area, model,track_ids, result):
    global captured
    global plate
    # Loop untuk setiap hasil deteksi
    for i, track_id in enumerate(track_ids):
        x1, y1, x2, y2, id, score, class_id = result[0].boxes.data[i].numpy()
        if track_id not in captured and int(class_id) != 2:
            capture_frame = detected_area[int(y1):int(y2), int(x1):int(x2)]
            license_detection = model(capture_frame)[0]
            for detection in license_detection.boxes.data.tolist():
                x1_license, y1_license, x2_license, y2_license, score_license, class_id_license = detection
                if class_id_license == 2:
                    license_frame = capture_frame[int(y1_license):int(y2_license), int(x1_license):int(x2_license)]
                    captured.append(track_id)
                    # Ganti fungsionalitas untuk menyimpan gambar, misalnya:
                    cv2.imwrite(f'./license_predict/{plate}.jpg', license_frame)
                    plate += 1

In [9]:
def resized_frame(frame):
    ## MENAMPILKAN LIVE
    # Mendapatkan dimensi asli frame
    original_height, original_width = frame.shape[:2]

    # Hitung faktor skala untuk menyesuaikan frame agar muat dalam satu layar laptop
    screen_height = 720  # Misalnya, resolusi layar laptop adalah 720p
    screen_width = 1280  # Misalnya, resolusi layar laptop adalah 1280p
    scale_factor = min(screen_height / original_height, screen_width / original_width)

    # Resize frame sesuai dengan faktor skala
    resized_frame = cv2.resize(frame, None, fx=scale_factor, fy=scale_factor)
    
    return resized_frame

In [10]:
ret = True

while ret:
    ret, frame = cap.read()
    
    if ret:
        detected_area, top_left_x, top_left_y, bottom_right_x, bottom_right_y = create_detection_frame(frame)
        
        frame_, track_ids, result, boxes = detect_objects(detected_area, model)
        
        if frame_ is not None:
            detect_license_plate(detected_area, model2, track_ids, result)

            # Pastikan untuk mengganti area yang dicrop dengan frame_
            frame[top_left_y:bottom_right_y, top_left_x:bottom_right_x] = frame_
            
            cv2.imshow('frame', resized_frame(frame))
            
            if cv2.waitKey(25) & 0xFF == ord('q'):
                break
        else:
            print("Model tidak mengembalikan hasil yang diharapkan. Lanjutkan ke frame berikutnya.")
            continue  # Lanjutkan ke frame berikutnya jika hasil tidak tersedia
        
cv2.destroyAllWindows()

requirements: Ultralytics requirement ['lapx>=0.5.2'] not found, attempting AutoUpdate...
   ---------------------------------------- 1.5/1.5 MB 885.7 kB/s eta 0:00:00
   ---------------------------------------- 2.8/2.8 MB 1.1 MB/s eta 0:00:0005

requirements: AutoUpdate success  8.8s, installed 1 package: ['lapx>=0.5.2']
requirements:  Restart runtime or rerun command for updates to take effect


0: 192x800 2 Mobils, 89.3ms
Speed: 4.5ms preprocess, 89.3ms inference, 792.8ms postprocess per image at shape (1, 3, 192, 800)
Model tidak mengembalikan hasil yang diharapkan. Lanjutkan ke frame berikutnya.

0: 192x800 2 Mobils, 75.3ms
Speed: 1.5ms preprocess, 75.3ms inference, 3.0ms postprocess per image at shape (1, 3, 192, 800)
Model tidak mengembalikan hasil yang diharapkan. Lanjutkan ke frame berikutnya.

0: 192x800 2 Mobils, 1 Plat Nomor, 69.9ms
Speed: 2.5ms preprocess, 69.9ms inference, 2.0ms postprocess per image at shape (1, 3, 192, 800)
Model tidak mengembalikan hasil yang diharapka